In [4]:
Pkg.add("Interact")
Pkg.add("Plots")

using JuMP # used for mathematical programming
using Interact # used for enabling the slider
using Plots # used for plotting 
gr();


INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of Interact
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Cloning cache of PlotUtils from https://github.com/JuliaPlots/PlotUtils.jl.git
INFO: Cloning cache of Plots from https://github.com/tbreloff/Plots.jl.git
INFO: Cloning cache of RecipesBase from https://github.com/JuliaPlots/RecipesBase.jl.git
INFO: Upgrading Conda: v0.3.2 => v0.4.0
INFO: Installing PlotUtils v0.1.1
INFO: Installing Plots v0.9.4
INFO: Installing RecipesBase v0.1.0
INFO: Building Conda
INFO: Building Plots
INFO: Cannot find deps/plotly-latest.min.js... downloading latest version.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1767k  100 1767k    0     0  1310k      0  0:00:01  0:00:01 --:--:-- 1311k
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the

Pkg.add("GR")
Pkg.build("GR")




INFO: To do a standard install of gr, copy and run this:



LoadError: LoadError: ArgumentError: Module GR not found in current path.
Run `Pkg.add("GR")` to install the GR package.
while loading In[4], in expression starting on line 7

In [ ]:
In [2]:
# Define some input data about the test system
# Maximum power output of generators
const g_max = [1000,1000];
# Minimum power output of generators
const g_min = [0,300];
# Incremental cost of generators 
const c_g = [50,100];
# Fixed cost of generators
const c_g0 = [1000,0]
# Incremental cost of wind generators
const c_w = 50;
# Total demand
const d = 1500;
# Wind forecast
const w_f = 200;

In [5]:
# In this cell we create  function solve_ed, which solves the economic dispatch problem for a given set of input parameters.
function solve_ed(g_max, g_min, c_g, c_w, d, w_f)
    #Define the economic dispatch (ED) model
    ed=Model() 
    
    # Define decision variables    
    @variable(ed, g_min[i] <= g[i=1:2] <= g_max[i]) # power output of generators
    @variable(ed, 0 <= w  <= w_f ) # wind power injection

    # Define the objective function
    @objective(ed,Min,sum(c_g[i] * g[i] for i=1:2)+ c_w * w)

    # Define the power balance constraint
    @constraint(ed, sum(g[i] for i=1:2) + w == d)

    # Solve statement
    solve(ed)
    
    # return the optimal value of the objective function and its minimizers
    return getvalue(g), getvalue(w), w_f-getvalue(w), getobjectivevalue(ed)
end

# Solve the economic dispatch problem
(g_opt,w_opt,ws_opt,obj)=solve_ed(g_max, g_min, c_g, c_w, d, w_f);

println("\n")
println("Dispatch of Generators: ", g_opt, " MW")
println("Dispatch of Wind: ", w_opt, " MW")
println("Wind spillage: ", w_f-w_opt, " MW") 
println("\n")
println("Total cost: ", obj, "\$")

LoadError: LoadError: UndefVarError: g_max not defined
while loading In[5], in expression starting on line 24

In [6]:
# This cell uses the package Interact defined above. 
# In this cell we create a manipulator that solves the economic dispatch problem for different values of c_g1_scale.

@manipulate for c_g1_scale = 0.5:0.01:3.0
    c_g_scale = [c_g[1]*c_g1_scale, c_g[2]] # update the incremental cost of the first generator at every iteration
    g_opt,w_opt,ws_opt,obj = solve_ed(g_max, g_min, c_g_scale, c_w, d, w_f) # solve the ed problem with the updated incremental cost
    HTML("Dispatch of Generators, MW: $(g_opt[:])<br>"*
    "Dispatch of Wind, MW: $w_opt<br>"*
    "Spillage of Wind, MW: $ws_opt<br>"*
    "Total cost, \$: $obj")
end

Interact.Slider{Float64}(Signal{Float64}(1.75, nactions=0),"c_g1_scale",1.75,0.5:0.01:3.0,".3f",true)

LoadError: LoadError: UndefVarError: c_g not defined
while loading In[6], in expression starting on line 4

In [7]:
@manipulate for c_w_scale = 1:0.1:3.5
    # Define the vectors of outputs
    obj_out = Float64[] 
    w_out = Float64[]
    g1_out = Float64[]
    g2_out = Float64[]
    
    for c_g1_scale = 0.5:0.01:3.0
        c_g_scale = [c_g[1]*c_g1_scale  c_g[2]] # update the incremental cost of the first generator at every iteration
        g_opt,w_opt,ws_opt,obj = solve_ed(g_max, g_min, c_g_scale, c_w_scale*c_w, d, w_f) # solve the ed problem with the updated incremental cost
        # Add the solution of the economic dispatch problem to the respective vectors
        push!(obj_out,obj)
        push!(w_out,w_opt)
        push!(g1_out,g_opt[1])
        push!(g2_out,g_opt[2])
    end
    
    # Plot the outputs
    
    # Plot the total cost
    p1 = plot(0.5:0.01:3.0,obj_out, xlabel="c_g1_scale", ylabel="Total cost, \$", ylims=(50000,200000),legend=nothing)
    # Plot the power output of Generator 1
    p2 = plot(0.5:0.01:3.0,g1_out, xlabel="c_g1_scale", ylabel="Dispatch of G1, MW", ylims=(0,1100),legend=nothing)
    # Plot the power output of Generator 2
    p3 = plot(0.5:0.01:3.0,g2_out, xlabel="c_g1_scale", ylabel="Dispatch of G2, MW", ylims=(0,1600),legend=nothing)
    # Plot the wind power output
    p4 = plot(0.5:0.01:3.0,w_out, xlabel="c_g1_scale", ylabel="Dispatch of Wind, MW", ylims=(-5,250),legend=nothing)
    plot(p1,p2,p3,p4)
end

Interact.Slider{Float64}(Signal{Float64}(2.2, nactions=0),"c_w_scale",2.2,1.0:0.1:3.5,".3f",true)

LoadError: LoadError: UndefVarError: c_g not defined
while loading In[7], in expression starting on line 1

In [8]:
@manipulate for demandscale = 0.2:0.01:1.5
    g_opt,w_opt,ws_opt,obj = solve_ed(g_max, g_min, c_g, c_w, demandscale*d, w_f)
    
    HTML("Dispatch of Generators, MW: $(g_opt[:])<br>"*
    "Dispatch of Wind, MW: $w_opt<br>"*
    "Spillage of Wind, MW: $ws_opt<br>"*
    "Total cost, \$: $obj")
end

Interact.Slider{Float64}(Signal{Float64}(0.85, nactions=0),"demandscale",0.85,0.2:0.01:1.5,".3f",true)

LoadError: LoadError: UndefVarError: g_max not defined
while loading In[8], in expression starting on line 1

In [ ]:
# In this cell we introduce binary decision u to the economic dispatch problem (function solve_ed)
function solve_uc(g_max, g_min, c_g, c_w, d, w_f)
    #Define the unit commitment (UC) model
    uc=Model() 
    
    # Define decision variables    
    @variable(uc, 0 <= g[i=1:2] <= g_max[i]) # power output of generators
    @variable(uc, u[i=1:2], Bin) # Binary status of generators
    @variable(uc, 0 <= w  <= w_f ) # wind power injection

    # Define the objective function
    @objective(uc,Min,sum(c_g[i] * g[i] for i=1:2)+ c_w * w)

    # Define the constraint on the maximum and minimum power output of each generator
    for i in 1:2
        @constraint(uc,  g[i] <= g_max[i] * u[i]) #maximum
        @constraint(uc,  g[i] >= g_min[i] * u[i]) #minimum
    end

    # Define the power balance constraint
    @constraint(uc, sum(g[i] for i=1:2) + w == d)

    # Solve statement
    status = solve(uc)
    
    return status, getvalue(g), getvalue(w), w_f-getvalue(w), getvalue(u), getobjectivevalue(uc)
end

# Solve the economic dispatch problem
status,g_opt,w_opt,ws_opt,u_opt,obj=solve_uc(g_max, g_min, c_g, c_w, d, w_f);

  
println("\n")
println("Dispatch of Generators: ", g_opt, " MW")
println("Commitments of Generators: ", u_opt)
println("Dispatch of Wind: ", w_opt, " MW")
println("Wind spillage: ", w_f-w_opt, " MW") 
println("\n")
println("Total cost: ", obj, "\$")

In [ ]:
@manipulate for demandscale = 0.2:0.01:1.5
    status, g_opt,w_opt,ws_opt, u_opt, obj = solve_uc(g_max, g_min, c_g, c_w, demandscale*d, w_f)
 
    if status == :Optimal
        HTML("Commitment of Generators, MW: $(u_opt[:])<br>"*
    "Dispatch of Generators, MW: $(g_opt[:])<br>"*
    "Dispatch of Wind, MW: $w_opt<br>"*
    "Spillage of Wind, MW: $ws_opt<br>"*
    "Total cost, \$: $obj")
    else
        HTML("Status: $status")
    end
end

In [9]:
@manipulate for demandscale = 0.2:0.05:1.45
    w_out = Float64[]
    g1_out = Float64[]
    g2_out = Float64[]
    
    for w_f_scale = 0.5:0.05:5
        status, g_opt,w_opt,ws_opt, u_opt, obj = solve_uc(g_max, g_min, c_g, c_w, demandscale*d, w_f*w_f_scale)
        push!(g1_out,g_opt[1])
        push!(g2_out,g_opt[2])
        push!(w_out,w_opt)
    end
    
    # Plot the power output of Generator 1
    p1 = plot(0.5:0.05:2,g1_out, xlabel="w_f_scale", ylabel="Dispatch of G1, MW", ylims=(-10,1000),legend=nothing)
    # Plot the power output of Generator 2  
    p2 = plot(0.5:0.05:2,g2_out, xlabel="w_f_scale", ylabel="Dispatch of G2, MW", ylims=(-10,1000),legend=nothing)
    # Plot the wind power output
    p3 = plot(0.5:0.05:2,w_out, xlabel="w_f_scale", ylabel="Dispatch of Wind, MW",ylims=(-10,500),legend=nothing)
    plot(p1,p2,p3)
end

Interact.Slider{Float64}(Signal{Float64}(0.8, nactions=0),"demandscale",0.8,0.2:0.05:1.45,".3f",true)

LoadError: LoadError: UndefVarError: solve_uc not defined
while loading In[9], in expression starting on line 1

In [10]:
g_max[2] = 1500
# Generator map (card(I) x card(b))
g_map = [0 1 0; 0 0 1];
# Generator map (card(I) x card(b))
w_map = [1 0 0; 0 1 0];
# Line map (card(L) x card(B))
f_map = [1 -1 0; 0 1 -1];
# the physical line start at the first node or something...
# Power flow limits (card(L))
f_max = [100,1000];
# Line impedance 
x = [0.001,0.001];
# Wind forecast (card(V))
w_f_vec = [150,50];
# Total demand (card(B))
d_vec = [0,0,1500];

LoadError: LoadError: UndefVarError: g_max not defined
while loading In[10], in expression starting on line 1

In [11]:
using GraphViz # Linux and OS X only

LoadError: LoadError: ArgumentError: Module GraphViz not found in current path.
Run `Pkg.add("GraphViz")` to install the GraphViz package.
while loading In[11], in expression starting on line 1

In [12]:
function writeDot(name, busIdx, busInj, renGen, f, t, lineFlow, lineLim, size=(11,14))
    # This function generates a graph that richly expresses the RTS96 system state.
    # name              a name for the graph and resulting dot file
    # busIdx            bus names (could be text) in order of busInj
    # busInj            injection at each bus
    # renGen            renewable generation at each bus (0 for non-wind buses)
    # f                 "from" node for each line
    # t                 "to" node for each line
    # lineFlow          flow on each line
    # lineLim           limit for each line
    # size              size of graphical output

    busInj = round(busInj,2)
    lineFlow = round(lineFlow,2)
    
    # Open the dot file, overwriting anything already there:
    dotfile = IOBuffer()
    
    # Begin writing the dot file:
    write(dotfile, "digraph $(name) {\nrankdir=LR;\n")

    # Set graph properties:
    write(dotfile, 
    "graph [fontname=helvetica, tooltip=\" \", overlap=false, size=\"$(size[1]),$(size[2])\", ratio=fill, orientation=\"portrait\",layout=dot];\n")

    # Set default node properties:
    write(dotfile, "node [fontname=helvetica, shape=square, style=filled, fontsize=20, color=\"#bdbdbd\"];\n")

    # Set default edge properties:
    write(dotfile, "edge [fontname=helvetica, style=\"setlinewidth(5)\"];\n")

    # Write bus data to dot file:
    for i = 1:length(busIdx)
        write(dotfile, 
        "$(i) [label=$(Int(busIdx[i])), tooltip=\"Inj = $(busInj[i])\"") # bus label and tooltip

        # Represent renewable nodes with blue circles:
        if union(find(renGen),i) == find(renGen)
            write(dotfile, ", shape=circle, color=\"#5677fc\"")
        end

        write(dotfile, "];\n")
    end

    # Write line data to file:

    for i = 1:length(f)

        normStrain = abs(lineFlow[i])/lineLim[i] # normalized strain on line i

        # Use flow direction to determine arrow direction,
        # label with flow,
        # and color according to strain
        if lineFlow[i] > 0
            write(dotfile, 
            "$(f[i]) -> $(t[i]) [label=$(lineFlow[i])")
        else
            write(dotfile, 
            "$(t[i]) -> $(f[i]) [label=$(-lineFlow[i])")
        end
        write(dotfile,
        ", tooltip=\" \", labeltooltip=\"Flow = $(round(Int,(normStrain*100)))%\", color=\"$(abs(round((1 - normStrain)/3,3))) 1.000 0.700\"];\n")
    end

    write(dotfile, "}\n")

    dottext = takebuf_string(dotfile)
    #print(dottext)

    return Graph(dottext)
end

writeDot (generic function with 2 methods)

In [ ]:
@manipulate for line2_limit in 1000:10:1500
#line2_limit
    f_max[2] = line2_limit
    #Define the optimal power flow (OPF) model
    opf=Model() 
    
    # Define decision variables
    @variables(opf, begin
        g[1:2] >= 0
        w[1:2] >= 0
        f[1:2]
        theta[1:3]
    end)

    # Define the objective function
    @objective(opf,Min,sum(c_g[i] * g[i] for i=1:2) + sum(c_w * w[v] for v=1:2))

    for i in 1:2
        @constraint(opf,  g[i] <= g_max[i] ) #maximum
        @constraint(opf,  g[i] >= g_min[i] ) #minimum
    end

    # Define the constraint on the wind power injections
    for v in 1:2
        @constraint(opf,  w[v] <= w_f_vec[v]); 
    end

    # Define the constraint on the power flows
    for l in 1:2
        @constraint(opf, f[l] <= f_max[l])   # direct flows
        @constraint(opf, f[l] >=  -f_max[l]) # reverse flows
    end

    # Define the power balance constraint
            # at each node, the power generated and flows in = what flows out
    for b in 1:3
        @constraint(opf, sum(g_map[i,b]* g[i] for i=1:2) + sum(w_map[v,b] * w[v] for v=1:2) + sum(f_map[l,b] * f[l] for l=1:2)>= d_vec[b]); 
    end

    # Calculate f[l]
    # The flow in any line is proportional to potential...
    for l in 1:2
        @constraint(opf, f[l] == 1/x[l] * sum(f_map[l,b] * theta[b] for b=1:3))   # power flow in every line
    end

    # Slack bus 
    @constraint(opf, theta[1] == 0)  # direct flows

    # Solve statement
    solve(opf)
    writeDot("UC",[1,2,3],[0.0;getvalue(g)],[getvalue(w);0.0],[2,3],[1,2],getvalue(f),f_max,(3,3))
end